In [ ]:
tickers = ['2330','2317','2454']
start = '2023-01-01'
end = '2023-12-31'
# IR0001 為加權股價報酬指數，通常用來當作回測 benchmark。
os.environ['mdate'] = start + ' ' + end
os.environ['ticker'] = ' '.join(tickers) + ' ' + 'IR0001'

!zipline ingest -b tquant

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipline.api import *
from zipline import run_algorithm
from zipline.utils.run_algo import get_transaction_detail
from zipline.api import *
from zipline.finance import commission, slippage

# tej_key
tej_key = 'AS9qRIEHRL2JaBWYq7WJtP9PulIhTO'
api_base = 'https://api.tej.com.tw'

os.environ['TEJAPI_KEY'] = tej_key
os.environ['TEJAPI_BASE'] = api_base

In [ ]:
def initialize(context):
    context.day = 0
    context.tickers = symbol('2330')
    set_slippage(slippage.VolumeShareSlippage(volume_limit=0.025, price_impact=0.1))
    set_commission(commission.Custom_TW_Commission(min_trade_cost=20, discount=1.0, tax=0.003))
    set_benchmark(symbol('IR0001'))

    # 設定可買開關
    context.can_buy = True
    # 買賣日期區間（UTC 時區）
    context.tradetime = pd.date_range(start='2023-06-25', end='2023-06-30', freq='D', tz='UTC')

def handle_data(context, data):
    today = get_datetime().normalize()  # 改用 Zipline 內建時間 API

    # 買進條件
    if context.can_buy and (today in context.tradetime):
        print("買進日:", today)
        order(context.tickers, 1000)
        context.can_buy = False

    # 持有期間天數累加
    if context.portfolio.positions[context.tickers].amount > 0:
        context.day += 1

    # 賣出條件
    if context.day > 180:
        order_target(context.tickers, 0)

# 模擬期間
start = '2023-01-01'
end = '2023-12-31'
start_date = pd.Timestamp(start, tz='utc')
end_date = pd.Timestamp(end, tz='utc')
capital_base = 1000000

results = run_algorithm(
    start=start_date,
    end=end_date,
    initialize=initialize,
    handle_data=handle_data,
    bundle='tquant',
    capital_base=capital_base,
)

results


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
!wget -O MicrosoftJhengHei.ttf https://github.com/a7532ariel/ms-web/raw/master/Microsoft-JhengHei.ttf
!wget -O ArialUnicodeMS.ttf https://github.com/texttechnologylab/DHd2019BoA/raw/master/fonts/Arial%20Unicode%20MS.TTF

fm.fontManager.addfont('MicrosoftJhengHei.ttf')
matplotlib.rc('font', family='Microsoft Jheng Hei')

matplotlib.font_manager.fontManager.addfont('ArialUnicodeMS.ttf')
matplotlib.rc('font', family='Arial Unicode MS')

from pyfolio.utils import extract_rets_pos_txn_from_zipline
import pyfolio

returns, positions, transactions = extract_rets_pos_txn_from_zipline(results)  # 從 results 資料表中取出 returns, positions & transactions
benchmark_rets = results.benchmark_return  # 取出 benchmark 的報酬率

# 繪製 Pyfolio 中提供的所有圖表

pyfolio.tears.create_full_tear_sheet(returns=returns,
                  positions=positions,
                  transactions=transactions,
                  benchmark_rets=benchmark_rets
                  )